In [1]:
import sys,os
PROJECT_ROOT=os.path.abspath("..")
SRC_PATH=os.path.join(PROJECT_ROOT,"src")
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

In [2]:
import pandas as pd
import numpy as np
train=pd.read_excel("../data/raw/train.xlsx").drop(columns=["date"])
test =pd.read_excel("../data/raw/test2.xlsx").drop(columns=["date"])

X_img_train=pd.read_csv("../outputs/train_img_features.csv").select_dtypes("number").values
X_img_test=pd.read_csv("../outputs/test_img_features.csv").select_dtypes("number").values

X_tab_train=train.select_dtypes("number").drop(columns=["price","id"], errors="ignore").values
X_tab_test=test.select_dtypes("number").drop(columns=["id"], errors="ignore").values

y_train=train["price"].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
X_tab=train.drop(columns=["price", "id"],errors="ignore").select_dtypes("number").values
y=train["price"].values
X_tr_tab,X_val_tab,y_tr_tab,y_val_tab=train_test_split(X_tab,y,test_size=0.2,random_state=42)
rf_tab=RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)
rf_tab.fit(X_tr_tab,y_tr_tab)
val_preds_tab=rf_tab.predict(X_val_tab)
print("Tabular Baseline Validation R²:",r2_score(y_val_tab, val_preds_tab))
print("Tabular Baseline Validation RMSE:",np.sqrt(mean_squared_error(y_val_tab, val_preds_tab)))
test_preds_tab=rf_tab.predict(test.drop(columns=["id"],errors="ignore").select_dtypes("number").values)
submission_tabular=pd.DataFrame({
    "id":test["id"],
    "predicted_price":test_preds_tab
})
submission_tabular.to_csv("../outputs/tabular_baseline.csv",index=False)

Tabular Baseline Validation R²: 0.8674217360010409
Tabular Baseline Validation RMSE: 128984.73593920104
Saved tabular baseline CSV → outputs/tabular_baseline.csv | shape: (5404, 2)


In [3]:
from sklearn.decomposition import PCA
pca=PCA(n_components=50, random_state=42)
X_img_train_pca=pca.fit_transform(X_img_train)
X_img_test_pca=pca.transform(X_img_test)

In [4]:
X_train=np.hstack([X_tab_train,X_img_train_pca])
X_test=np.hstack([X_tab_test,X_img_test_pca])
print("Fused Train Shape:",X_train.shape)
print("Fused Test Shape:",X_test.shape)

Fused Train Shape: (16209, 68)
Fused Test Shape: (5404, 68)


In [5]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val=train_test_split(
    X_train,y_train,test_size=0.2,random_state=42
)

In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np
xgb=XGBRegressor(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1,
    reg_lambda=1,
    random_state=42,
    n_jobs=-1
)
xgb.fit(X_tr,y_tr)
val_preds=xgb.predict(X_val)
r2=r2_score(y_val,val_preds)
rmse=np.sqrt(mean_squared_error(y_val,val_preds))
print("Validation R²:",r2)
print("Validation RMSE:",rmse)

Validation R²: 0.8855860233306885
Validation RMSE: 119823.42635728624


In [ ]:
test_preds=xgb.predict(X_test)
print("Test prediction shape:",test_preds.shape)
submission=pd.DataFrame({
    "id":test["id"],
    "predicted_price":test_preds
})
submission.to_csv("../outputs/submission.csv",index=False)

Test prediction shape: (5404,)
